In [1]:
!pip3 install transformers

In [2]:
!pip3 install torch

In [3]:

import pandas as pd
from transformers import BertTokenizer, BertModel
import torch
import numpy as np
        
# Load data from the CSV file
data = pd.read_csv('processed_data.csv')

# Load BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Function to encode text into BERT embeddings
def encode_text(text):
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=128)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state[:, 0, :].squeeze().numpy()

# Apply the encoding to the 'conditions' column to generate embeddings
data['Interventions_embeddings'] = data['Interventions'].apply(encode_text)



/Users/priteshpunj/anaconda3/lib/python3.11/site-packages/transformers/utils/generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
/Users/priteshpunj/anaconda3/lib/python3.11/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

KeyError: 'embeddings'

In [4]:
# Convert the embeddings from the 'embeddings' column into a 2D NumPy array
embeddings = np.vstack(data['Interventions_embeddings'].values)

# Check the shape of the embeddings
print(embeddings.shape)  # This will show the number of samples and the size of each embedding vector

(20586, 768)


In [5]:
# Convert embeddings from numpy arrays to strings (if needed) for saving to CSV
data['Interventions_embeddings'] = data['Interventions_embeddings'].apply(lambda x: ' '.join(map(str, x)))

# Save the DataFrame to a new CSV file
data.to_csv('processed_data_with_embeddings.csv', index=False)